In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2024-challenge1/sample_submission.csv
/kaggle/input/widsdatathon2024-challenge1/training.csv
/kaggle/input/widsdatathon2024-challenge1/test.csv


In [23]:
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import optuna
from category_encoders import OneHotEncoder, MEstimateEncoder, CatBoostEncoder, OrdinalEncoder
from sklearn import set_config
import category_encoders
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.preprocessing import FunctionTransformer,StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.metrics import auc, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_rows', 150)

# visualizations

import missingno as msno 
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

/kaggle/input/widsdatathon2024-challenge1/sample_submission.csv
/kaggle/input/widsdatathon2024-challenge1/training.csv
/kaggle/input/widsdatathon2024-challenge1/test.csv


In [24]:
train = pd.read_csv(r'/kaggle/input/widsdatathon2024-challenge1/training.csv', index_col='patient_id')
test = pd.read_csv(r'/kaggle/input/widsdatathon2024-challenge1/test.csv', index_col='patient_id')
sub  = pd.read_csv(r'/kaggle/input/widsdatathon2024-challenge1/sample_submission.csv')

In [ ]:
train.select_dtypes(include=['object']).describe().T

In [ ]:
train.isnull().sum()

In [ ]:
train.notna().sum()

In [ ]:
train.head()

In [25]:
drop_cols = ['patient_gender']

train.drop(drop_cols,axis=1,inplace=True)
test.drop(drop_cols,axis=1,inplace=True)

In [26]:
train.select_dtypes(include=['object']).describe().T

,count,unique,top,freq
patient_race,6521,5,White,3588
payer_type,11103,3,COMMERCIAL,6032
patient_state,12855,50,CA,2438
breast_cancer_diagnosis_code,12906,50,1749,1982
breast_cancer_diagnosis_desc,12906,50,"Malignant neoplasm of breast (female), unspeci...",1982
metastatic_cancer_diagnosis_code,12906,43,C773,7052
metastatic_first_novel_treatment,24,2,PEMBROLIZUMAB,13
metastatic_first_novel_treatment_type,24,1,Antineoplastics,24
Region,12854,4,South,3919
Division,12854,9,East North Central,2923


In [ ]:
train.describe().T.style.background_gradient()

In [ ]:
# one-hot encoding

# how to deal w missing values for race and payer type?

# fill nulls w/ mean (can also try median) for bmi; the avg bmi is ~the avg American woman's BMI

# metastatic_first_novel_treatment, and metastatic_first_novel_treatment_type
# ^ have barely any non-null values (around 1% that are non-null), so we can just drop those cols

In [30]:
# TRY: filling BMI nulls with mean BMI

bmi_mean = train['bmi'].mean()
bmi_mean2 = test['bmi'].mean()

train['bmi'] = train['bmi'].fillna(bmi_mean)
test['bmi'] = test['bmi'].fillna(bmi_mean2)

In [31]:
# DROP: metastatic_first_novel_treatment, metastatic_first_novel_treatment_type

drop_cols = ['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type']

train.drop(drop_cols,axis=1,inplace=True)
test.drop(drop_cols,axis=1,inplace=True)

KeyError: "['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type'] not found in axis"

In [8]:
# FOR NOW: drop race, payer type

drop_cols = ['patient_race', 'payer_type']
train.drop(drop_cols,axis=1,inplace=True)
test.drop(drop_cols,axis=1,inplace=True)

In [ ]:
train.info()

In [9]:
train.dropna()
test.dropna()

train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12906 entries, 475714 to 179597
Data columns (total 77 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   patient_state                     12855 non-null  object 
 1   patient_zip3                      12906 non-null  int64  
 2   patient_age                       12906 non-null  int64  
 3   bmi                               12906 non-null  float64
 4   breast_cancer_diagnosis_code      12906 non-null  object 
 5   breast_cancer_diagnosis_desc      12906 non-null  object 
 6   metastatic_cancer_diagnosis_code  12906 non-null  object 
 7   Region                            12854 non-null  object 
 8   Division                          12854 non-null  object 
 9   population                        12905 non-null  float64
 10  density                           12905 non-null  float64
 11  age_median                        12905 non-null  float64
 12  age

In [10]:
train.dropna(inplace=True)
test.dropna(inplace=True)

train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12822 entries, 475714 to 179597
Data columns (total 77 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   patient_state                     12822 non-null  object 
 1   patient_zip3                      12822 non-null  int64  
 2   patient_age                       12822 non-null  int64  
 3   bmi                               12822 non-null  float64
 4   breast_cancer_diagnosis_code      12822 non-null  object 
 5   breast_cancer_diagnosis_desc      12822 non-null  object 
 6   metastatic_cancer_diagnosis_code  12822 non-null  object 
 7   Region                            12822 non-null  object 
 8   Division                          12822 non-null  object 
 9   population                        12822 non-null  float64
 10  density                           12822 non-null  float64
 11  age_median                        12822 non-null  float64
 12  age

In [ ]:
# NOW TRY: one-hot encoding for patient race, payer type, as well as all other categorical variables
# This must be done AFTER dealing with all nulls

train = pd.get_dummies(train)
test = pd.get_dummies(test)

train.head(10)

In [ ]:
train.info()

In [21]:
import math	

#columns = [columns in train if dtypes==bool else ignore]
#train.columns = train.columns.replace({True: 1, False: 0})
# inplace=True
for column in train.columns:
    if train[column].dtype == 'bool':
        train[column] = int(train[column] == True)
    if train[column].dtype == 'float':
        train[column] = train[column].astype(np.int64)
#     elif train[column].dtype == 'ob':
#         train[column] = str(train[column])
        # df['int_column'] = df['float_column'].astype(np.int64)
#     if train[column].dtype == 'int':
#         train[column] = train[column].astype(float)
#         train[column] = train[column].astype(str)

for column in test.columns:
    if test[column].dtype == 'bool':
        test[column] = int(test[column] == True)
    if test[column].dtype == 'float':
        test[column] = test[column].astype(np.int64)
#     else:
#         test[column] = str(test[column])
#     if test[column].dtype == 'int':
#         test[column] = test[column].astype(float)
# for column in train.columns:
#     if train[column].dtype == 'bool':
#         for item in train.columns:
# #             print(item)
#             if item is True:
#                 item = 1
#             elif item is False:
#                 item = 0
            
#         train = train.apply(pd.to_numeric)
        #print('found')
#         train[column] = pd.to_numeric(train[column], errors='raise')

        
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12822 entries, 475714 to 179597
Data columns (total 77 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   patient_state                     12822 non-null  object
 1   patient_zip3                      12822 non-null  object
 2   patient_age                       12822 non-null  object
 3   bmi                               12822 non-null  object
 4   breast_cancer_diagnosis_code      12822 non-null  object
 5   breast_cancer_diagnosis_desc      12822 non-null  object
 6   metastatic_cancer_diagnosis_code  12822 non-null  object
 7   Region                            12822 non-null  object
 8   Division                          12822 non-null  object
 9   population                        12822 non-null  object
 10  density                           12822 non-null  object
 11  age_median                        12822 non-null  object
 12  age_under_10     

In [19]:
train.head()

,patient_state,patient_zip3,patient_age,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,Region,Division,population,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
patient_id,,,,,,,,,,,,,,,,,,,,,
475714,CA,924,84,28,C50919,Malignant neoplasm of unsp site of unspecified...,C7989,West,Pacific,31437,...,12,22,10,27,11,3,52,8,18,1
349367,CA,928,62,28,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,West,Pacific,39121,...,8,10,8,30,7,4,42,8,20,1
138632,TX,760,43,38,C50112,Malignant neoplasm of central portion of left ...,C773,South,West South Central,21996,...,11,9,3,31,15,7,40,7,14,1
617843,CA,926,45,28,C50212,Malig neoplasm of upper-inner quadrant of left...,C773,West,Pacific,32795,...,8,8,5,27,4,4,42,7,15,0
817482,ID,836,55,28,1749,"Malignant neoplasm of breast (female), unspeci...",C773,West,Mountain,10886,...,15,11,1,26,12,13,41,4,11,0


In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
train_x= train.drop(['DiagPeriodL90D'], axis=1)
train_y = train['DiagPeriodL90D']

In [ ]:
cat_features=train_x.select_dtypes(include=['int']).columns.tolist()
model=CatBoostClassifier(cat_features=cat_features,task_type='GPU')
model.fit(train_x,train_y)

In [ ]:
class_predictions = model.predict(train_x)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, random_state=1618)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(train_x, train_y)

predictions = model.predict(val_x)

In [ ]:
mse = ((predictions - val_y) ** 2).mean()
print("Mean Squared Error:", mse)

from sklearn.metrics import r2_score

r2 = r2_score(val_y, predictions)
print("R-squared:", r2)